In [1]:
# !pip install git+https://github.com/NHERI-SimCenter/BrailsPlusPlus
# !pip install plotly

# Example 1 - Atlantic county example using the previous R2D inputs

In [2]:
# Written: sy Aug 2024
# License: BSD-2

"""
 Purpose: Testing Imputer
"""

import os
import sys
import copy
import json

import numpy as np

sys.path.insert(0, "../../")
from brails.utils import Importer
from brails.types.image_set import ImageSet    
from brails.types.asset_inventory import Asset, AssetInventory


INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [3]:
file_path = "./nsi_altantic_minimal.csv"

In [4]:
# create an Import to get the classes

importer = Importer()
knn_imputer_class = importer.get_class("KnnImputer")

In [5]:
inventory = AssetInventory()
inventory.read_from_csv(file_path,keep_existing=True,id_column='id') 

No existing inventory found. Reading in the new inventory from the file.


True

In [6]:
inventory.get_asset_features(4)[1]  # empty or 'NA' are missing values

{'id': 4,
 'City': 'Somers Point',
 'OccupancyClass': 'RES1',
 'BuildingMaterial': 'Wood',
 'DesignLevel': 'NE',
 'YearBuilt': 1994,
 'NumberOfStories': 2,
 'PlanArea': 2416,
 'FoundationType': 3504,
 'SplitLevel': 0,
 'Garage': 0,
 'FirstFloorHt': 2,
 'FloodZone': 'X',
 'WindZone': 'I',
 'AvgJanTemp': 'Above',
 'RoofShape': 'hip',
 'RoofSlope': 0,
 'RoofFrameType': 'trs',
 'WindowArea': 0,
 'DesignWindSpeed': 121.5296499,
 'NumberOfUnits': 1,
 'MeanRoofHt': 35.04684071,
 'OBJECTID': 847181,
 'geometry': 'POINT (-74.61723667499923 39.30829801705697)',
 'type': 'building'}

In [7]:
#inventory.remove_features(['RoofShape'])

In [8]:
hurricaneInferer = importer.get_class("HazusHurricaneInferer")
inferer=hurricaneInferer(input_inventory=inventory,clean_features = True)

In [9]:
inventory.get_asset_coordinates(1)[1]

[[-74.61718179, 39.3079507]]

In [10]:
new_inventory = inferer.infer()

Done inference. It took 0.16 mins


In [11]:
new_inventory.get_asset_features(156)[1]  # empty or 'NA' are missing values

{'BuildingTag': 'W.SF.',
 'TerrainRoughness': 15,
 'NumberOfStories': 1,
 'RoofShape': 'gab',
 'SecondaryWaterResistance': 1,
 'Shutters': 0,
 'RoofDeckAttachmentW': '8d',
 'RoofToWallConnection': 'strap',
 'GarageDoor': 'no'}

In [12]:
# Make values human readable 

In [13]:
new_inventory.get_asset_features(100)[1]  # empty or 'NA' are missing values

{'BuildingTag': 'S.ECB.L',
 'TerrainRoughness': 15,
 'RoofCover': 'bur',
 'WindowAreaRatio': 'low',
 'RoofDeckAttachmentM': 'sup',
 'Shutters': 0,
 'WindDebrisClass': 'A'}

In [14]:
_ = new_inventory.write_to_geojson(
    output_file="test.json")

# Example 2 - Fort Myers Beach

In [15]:
LOCATION_NAME = 'Fort Myers Beach' #'Fort Myers Beach, FL'
INVENTORY_OUTPUT = 'FMB.geojson' #'FortMyersInventory_HU.geojson'
NO_POSSIBLE_WORLDS = 1

In [16]:
importer = Importer()

In [17]:
region_data = {"type": "locationName", "data": LOCATION_NAME}
region_boundary_class = importer.get_class("RegionBoundary")
region_boundary_object = region_boundary_class(region_data)

In [18]:
nsi_class = importer.get_class('NSI_Parser')
nsi = nsi_class()
nsi_inventory = nsi.get_raw_data_given_boundary(region_boundary_object, 'ft')


Searching for Fort Myers Beach...
Found Fort Myers Beach, Lee County, Florida, 33931, United States

Getting National Structure Inventory (NSI) building data for the entered location input...
Found a total of 3558 building points in NSI that are within the entered region of interest


In [19]:
scraper_class = importer.get_class('USA_FootprintScraper')
scraper = scraper_class({'length': 'ft'})
scraper_inventory = scraper.get_footprints(region_boundary_object)


Searching for Fort Myers Beach...
Found Fort Myers Beach, Lee County, Florida, 33931, United States

Meshing the defined area...

Meshing complete. Covered Fort Myers Beach with a single rectangular cell


100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1000.31it/s]



Found a total of 3102 building footprints in Fort Myers Beach


In [20]:
nsi_inventory = nsi.get_filtered_data_given_inventory(
    scraper_inventory, "ft", get_extended_features=True)


Getting National Structure Inventory (NSI) building data for the entered location input...
Found a total of 2797 building points in NSI that match the footprint data.


In [21]:
knn_imputer_class = importer.get_class("KnnImputer")

imputer = knn_imputer_class(
    nsi_inventory, n_possible_worlds=NO_POSSIBLE_WORLDS,
    exclude_features=['lat', 'lon', 'fd_id'])
imputed_inventory = imputer.impute()

Features with no reference data cannot be imputed. Removing them from the imputation target: buildingheight
Missing percentages among 3102 assets
fparea: 9.83%
numstories: 9.83%
erabuilt: 9.83%
repaircost: 9.83%
constype: 9.83%
occupancy: 9.83%
splitlevel: 9.83%
basement: 28.05%
Primitive imputation done.
Running the main imputation. This may take a while.
Done imputation. It took 0.01 mins


## Assigning feature temporarily

In [22]:
for key, val in imputed_inventory.inventory.items():
    val.add_features({"DesignWindSpeed":30,
                     "RoofShape":'hip',
                     "AvgJanTemp":'above',
                     "DesignLevel":'E'})

In [23]:
imputed_inventory.change_feature_names({'erabuilt': 'YearBuilt',
                                        'constype': 'BuildingMaterial',
                                        'fpAreas': 'PlanArea',
                                        'numstories': 'NumberOfStories',
                                        'occupancy': 'OccupancyClass'
                                       })
hurricaneInferer = importer.get_class("HazusHurricaneInferer")
inferer = hurricaneInferer(input_inventory=imputed_inventory, clean_features=True)
hazus_inferred_inventory = inferer.infer()

Done inference. It took 0.00 mins


In [24]:
hazus_inferred_inventory.inventory[125].features


{'BuildingTag': 'W.MUH.',
 'TerrainRoughness': 15,
 'RoofShape': 'hip',
 'NumberOfStories': 1,
 'RoofQuality': 'null',
 'Shutters': 0,
 'RoofToWallConnection': 'tnail',
 'RoofDeckAttachmentW': '8d',
 'RoofCover': 'null',
 'SecondaryWaterResistance': 1}

In [25]:
hazus_inferred_inventory.inventory[10].features


{'BuildingTag': 'W.SF.',
 'TerrainRoughness': 15,
 'NumberOfStories': 1,
 'RoofShape': 'hip',
 'SecondaryWaterResistance': 0,
 'Shutters': 0,
 'RoofDeckAttachmentW': '8d',
 'RoofToWallConnection': 'tnail',
 'GarageDoor': 'std'}